# Quantum Neural Tangent Kernel demo
This notebook shows how to compute and visualize the Q-NTK for a tiny quantum neural network alongside a classical NTK.

## Why NTK matters- The Neural Tangent Kernel (NTK) captures how parameter updates move model outputs near initialization.
- In very wide networks, gradient descent behaves like kernel regression with this NTK.
- The Quantum NTK (Q-NTK) applies the same idea to parameterized quantum circuits.

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from app.dataset import generate_regression_data
from app.qnn_model import init_qnn_params
from app.classical_model import init_classical_params
from app.qntk import compute_qntk_matrix, eigendecomposition as q_eig
from app.classical_ntk import compute_classical_ntk_matrix, eigendecomposition as c_eig
from app.dynamics import train_qnn_gd, train_classical_gd
from app.plots import plot_kernel_heatmap, plot_eigenvalues, plot_training_dynamics

X_train, X_test, y_train, y_test = generate_regression_data(n_points=12, seed=0)
q_params = init_qnn_params(seed=0)
c_params = init_classical_params(hidden_dim=6, seed=0)

Kq = compute_qntk_matrix(X_train, q_params)
Kc = compute_classical_ntk_matrix(X_train, c_params)

ev_q, _ = q_eig(Kq)
ev_c, _ = c_eig(Kc)

_, loss_q = train_qnn_gd(X_train, y_train, q_params, steps=20)
_, loss_c = train_classical_gd(X_train, y_train, c_params, steps=10)

sns.heatmap(Kq, cmap='viridis')
plt.title('Q-NTK heatmap')
plt.show()

sns.heatmap(Kc, cmap='magma')
plt.title('Classical NTK heatmap')
plt.show()

plt.semilogy(ev_q, label='Q-NTK eigenvalues')
plt.semilogy(ev_c, label='Classical NTK eigenvalues')
plt.legend()
plt.show()

plt.plot(loss_q, label='QNN loss')
plt.plot(loss_c, label='Classical loss')
plt.legend()
plt.xlabel('Step')
plt.ylabel('MSE')
plt.show()